In [1]:
# Data cleaning for CT Pretrial Detainees
# (Springboard Capstone 1)
# 2018, Misty M. Giles

import numpy as np
import pandas as pd

In [2]:
# Accused...csv is the full 260MB file with data to 11 November 2018.  Replace 
# with a smaller file ('detainees_2016_07.csv' suggested as it's already in the
# repo) if you want to re-run this notebook in a reasonable time.
file = 'Accused_Pre-Trial_Inmates_in_Correctional_Facilities.csv'
#file = 'detainees_2016_07.csv'

# Read data in.  Update the columns to lower case and replace spaces with _s.
df = pd.read_csv(file, parse_dates=['DOWNLOAD DATE', 'LATEST ADMISSION DATE'])
df.columns = ['download_date', 'identifier', 'latest_admission_date', 'race', 
              'gender', 'age', 'bond_amount', 'offense', 'facility', 'detainer']

In [3]:
# Check .info() to see if the dtypes are appropriate.
print(df.info())

# Check the first rows to ensure that the data is expected.
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2677123 entries, 0 to 2677122
Data columns (total 10 columns):
download_date            datetime64[ns]
identifier               object
latest_admission_date    datetime64[ns]
race                     object
gender                   object
age                      int64
bond_amount              int64
offense                  object
facility                 object
detainer                 object
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 204.2+ MB
None


,download_date,identifier,latest_admission_date,race,gender,age,bond_amount,offense,facility,detainer
0,2016-07-01,ZZEBBEWZ,2016-06-17,WHITE,M,63,5000,"ASSAULT, THIRD DEGREE AM",CORRIGAN CI,NONE
1,2016-07-01,ZZEBBRZC,2016-06-30,BLACK,M,42,500,"FAILURE TO APPEAR, SECOND DEGREE AM",HARTFORD CC,NONE
2,2016-07-01,ZZEBCBHC,2016-06-06,WHITE,M,60,40000,EVADING RESPONSIBILITY M,HARTFORD CC,NONE
3,2016-07-01,ZZEBCCWL,2016-04-25,HISPANIC,M,44,125000,CRIMINAL POSSESSION OF A PISTOL DF,HARTFORD CC,NONE
4,2016-07-01,ZZEBCHBZ,2016-06-21,HISPANIC,M,58,2500,BREACH OF PEACE BM,CORRIGAN CI,NONE


In [4]:
# Change race, gender, facility, and detainer to category type and verify.
# Categorical variables chosen to speed up the processing slightly.
df.race = df.race.astype('category')
df.gender = df.gender.astype('category')
df.facility = df.facility.astype('category')
df.detainer = df.detainer.astype('category')

# Two more variables are needed: one to tell us felony or misdemeanor and the classification
# (if available) and one to tell us the number of days detained per arrest.  Setup is below.

# Separate the last two letters of the offense to get the offense class, if classed.
# If offense isn't classed, use np.nan to easily separate them out later.
df['offense_class'] = [offense[38:] if offense.endswith(('AM','BM','CM','DM','UM',' M', 
                       'AF','BF','CF','DF','UF',' F')) else np.nan for offense in df.offense]
# Set offense_class to categorical.
df.offense_class = df.offense_class.astype('category')
# Do the time math for the number of days each person spent per arrest.  Because the state says
# dates of more than a year cannot be assumed correct, another column that should count the
# rows in which the identifier appears is added after 'days.'  
df['days'] = df.download_date - df.latest_admission_date
# An imperfect check to tell us if someone with an admission date > 2015 could have been
# in the system the number of days listed.  Doesn't work well with repeat offenders.
# It should count the number of times the identifier appears in the list.
df['counts'] = df.groupby(['identifier', 'latest_admission_date'])['identifier'].transform('count')

# Verify 13 columns (original 10 + 3 above) and variable types.
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2677123 entries, 0 to 2677122
Data columns (total 13 columns):
download_date            datetime64[ns]
identifier               object
latest_admission_date    datetime64[ns]
race                     category
gender                   category
age                      int64
bond_amount              int64
offense                  object
facility                 category
detainer                 category
offense_class            category
days                     timedelta64[ns]
counts                   int64
dtypes: category(5), datetime64[ns](2), int64(3), object(2), timedelta64[ns](1)
memory usage: 176.2+ MB
None


In [5]:
# Check the numbers of the categories against the number of expected values. 
print(df.race.nunique())
print(df.gender.nunique())
print(df.facility.nunique())
print(df.detainer.nunique())
print(df.offense_class.nunique())

# Check the expected values of the categories.  .tolist() used to show entire list as needed.
print(df.race.values.unique())
print(df.gender.values.unique())
print(df.facility.values.unique().tolist())
print(df.detainer.values.unique().tolist())
print(df.offense_class.values.unique().tolist())

# Check for any empty values.  Offense_class should contain some NaNs; other columns should not.
print(df.count())

5
2
50
9
11
[WHITE, BLACK, HISPANIC, ASIAN, AMER IND]
Categories (5, object): [WHITE, BLACK, HISPANIC, ASIAN, AMER IND]
[M, F]
Categories (2, object): [M, F]
['CORRIGAN CI', 'HARTFORD CC', 'YORK CI', 'NEW HAVEN CC', 'BRIDGEPORT CC', 'GARNER', 'MACDOUGALL', 'WALKER RC', 'NORTHERN CI', 'CHESHIRE CI', 'OSBORN CI', 'MANSON YI', 'MH-WHITING', 'FEDERAL MARSHAL', 'D KIMBALL HSP', 'ROBINSON CI', 'ST MARYS HOSP', 'MH-CVH', 'ENFIELD CI', 'HARTFORD HOSP', 'WILLARD-CYBULSKI CI', 'UCONN HOSP', 'ST VINC HOSP', 'LAWRENCE HOSP', 'WATERBRY HOSP', 'MEDICAL HOSP', 'YALE/N.H.HOSP', 'JOHNSON HOSP', 'MH-YALE/N.H.', 'MH-ST MARY', 'BROOKLYN CI', 'DANBURY HOSP', 'BACKUS HOSP', 'RADGOWSKI', 'ST FRAN HOSP', 'PO-GARIBALDI', 'BRIDGEPORT HOSP', 'PO-WILLIAMS', 'PO-CARRINGTON', '4YV', 'ST RAPH HOSP', 'MH-HTFD HOSP', 'CONSIGN @ NY', 'MH-ST FRANCIS', 'MH-UCONN', 'PO-FERRARO', 'NORWALK HOSP', 'MH-ST RAPHAEL', 'MID-STATE HOSP', 'MER-WALL HOSP']
['NONE', 'SPECIAL PAROLE', 'FEDERAL', 'STATE OF CT', 'OTHER STATE', 'IMMIGRAT

In [6]:
# For each admission per identifier, get the last row of the data.  This 
# row (essentially, max(download_date)) contains the information from the day that 
# the detainee receives a verdict, is bailed out, or had charges dropped.  
# This row was chosen purposely as the least likely to have errors.  Bond amounts can 
# change during negotiations or as charges are changed or dropped.  This is a simplification
# to help meet deadlines.
grouped_df = df.loc[df.groupby(['identifier', 'latest_admission_date'])['download_date'].idxmax()]

In [7]:
# Verify the number of detainees (repeat offenders count per arrest) and that
# the datatypes/counts are still as expected.  Offense_class is only value that should
# contain NaNs.
grouped_df.info()
grouped_df = grouped_df.sort_values(by='identifier')
grouped_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40147 entries, 2517248 to 730145
Data columns (total 13 columns):
download_date            40147 non-null datetime64[ns]
identifier               40147 non-null object
latest_admission_date    40147 non-null datetime64[ns]
race                     40147 non-null category
gender                   40147 non-null category
age                      40147 non-null int64
bond_amount              40147 non-null int64
offense                  40147 non-null object
facility                 40147 non-null category
detainer                 40147 non-null category
offense_class            28808 non-null category
days                     40147 non-null timedelta64[ns]
counts                   40147 non-null int64
dtypes: category(5), datetime64[ns](2), int64(3), object(2), timedelta64[ns](1)
memory usage: 3.0+ MB


,download_date,identifier,latest_admission_date,race,gender,age,bond_amount,offense,facility,detainer,offense_class,days,counts
2517248,2018-09-27,ZZEBBBCS,2018-08-21,WHITE,M,47,900000,"FAILURE TO APPEAR, FIRST DEGREE DF",NORTHERN CI,NONE,DF,37 days,37
248828,2016-10-21,ZZEBBBJW,2016-08-24,WHITE,M,44,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,58 days,26
1090690,2017-07-13,ZZEBBBJW,2017-07-06,WHITE,M,44,100000,CRIM VIOL OF PROTECTIVE ORDER AM,HARTFORD CC,NONE,AM,7 days,7
2676743,2018-11-11,ZZEBBBJW,2018-09-27,WHITE,M,46,50000,VIOLATION OF PROBATION OR COND DISCHG,HARTFORD CC,NONE,NaN,45 days,45
215673,2016-10-11,ZZEBBBSZ,2016-10-07,BLACK,F,57,2500,THREATENING AM,YORK CI,NONE,AM,4 days,3


In [8]:
# Separate out the arrests where felony/misdemeanor is stated.  Again, print
# out the info.  All values should be equal.  No NaNs expected here.
classed = grouped_df.loc[grouped_df['offense_class'].notnull()]
classed.info()
classed.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28808 entries, 2517248 to 730145
Data columns (total 13 columns):
download_date            28808 non-null datetime64[ns]
identifier               28808 non-null object
latest_admission_date    28808 non-null datetime64[ns]
race                     28808 non-null category
gender                   28808 non-null category
age                      28808 non-null int64
bond_amount              28808 non-null int64
offense                  28808 non-null object
facility                 28808 non-null category
detainer                 28808 non-null category
offense_class            28808 non-null category
days                     28808 non-null timedelta64[ns]
counts                   28808 non-null int64
dtypes: category(5), datetime64[ns](2), int64(3), object(2), timedelta64[ns](1)
memory usage: 2.1+ MB


,download_date,identifier,latest_admission_date,race,gender,age,bond_amount,offense,facility,detainer,offense_class,days,counts
2517248,2018-09-27,ZZEBBBCS,2018-08-21,WHITE,M,47,900000,"FAILURE TO APPEAR, FIRST DEGREE DF",NORTHERN CI,NONE,DF,37 days,37
248828,2016-10-21,ZZEBBBJW,2016-08-24,WHITE,M,44,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,58 days,26
1090690,2017-07-13,ZZEBBBJW,2017-07-06,WHITE,M,44,100000,CRIM VIOL OF PROTECTIVE ORDER AM,HARTFORD CC,NONE,AM,7 days,7
215673,2016-10-11,ZZEBBBSZ,2016-10-07,BLACK,F,57,2500,THREATENING AM,YORK CI,NONE,AM,4 days,3
1296934,2017-09-13,ZZEBBBZJ,2017-09-12,BLACK,M,43,20000,THREATENING AM,BRIDGEPORT CC,NONE,AM,1 days,1


In [9]:
# Separate out the arrests without felony/misdemeanor classifications.
# These are probably not useful for this project, but we'll retain them 
# just in case.
not_classed = grouped_df.loc[grouped_df['offense_class'].isnull()]
not_classed.info()
not_classed.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11339 entries, 2676743 to 1858043
Data columns (total 13 columns):
download_date            11339 non-null datetime64[ns]
identifier               11339 non-null object
latest_admission_date    11339 non-null datetime64[ns]
race                     11339 non-null category
gender                   11339 non-null category
age                      11339 non-null int64
bond_amount              11339 non-null int64
offense                  11339 non-null object
facility                 11339 non-null category
detainer                 11339 non-null category
offense_class            0 non-null category
days                     11339 non-null timedelta64[ns]
counts                   11339 non-null int64
dtypes: category(5), datetime64[ns](2), int64(3), object(2), timedelta64[ns](1)
memory usage: 856.6+ KB


,download_date,identifier,latest_admission_date,race,gender,age,bond_amount,offense,facility,detainer,offense_class,days,counts
2676743,2018-11-11,ZZEBBBJW,2018-09-27,WHITE,M,46,50000,VIOLATION OF PROBATION OR COND DISCHG,HARTFORD CC,NONE,NaN,45 days,45
655646,2017-03-01,ZZEBBBZL,2017-02-14,BLACK,M,44,1000,PROHIB ACTS RE: DRUG PARAPHERNALIA,BRIDGEPORT CC,NONE,NaN,15 days,15
2075254,2018-05-22,ZZEBBEWB,2018-03-16,BLACK,M,48,25000,VIOLATION OF PROBATION OR COND DISCHG,BRIDGEPORT CC,NONE,NaN,67 days,64
2441831,2018-09-06,ZZEBBEWB,2018-08-16,BLACK,M,48,3050,PROHIB ACTS RE: DRUG PARAPHERNALIA,BRIDGEPORT CC,NONE,NaN,21 days,21
1081131,2017-07-11,ZZEBBHJE,2017-07-10,BLACK,M,50,900,VIOLATION OF PROBATION OR COND DISCHG,BRIDGEPORT CC,NONE,NaN,1 days,1


In [10]:
# Just for fun, check over the basic stats and see what we'll be working with later.
# CT says that bonds under 100 can be considered placeholders, but I don't want to drop them yet.
print(classed.bond_amount.max(), 'max bond')
print(classed.bond_amount.min(), 'min bond')
print(classed.bond_amount.loc[classed.bond_amount < 100].count(), 'out of', len(classed), 'have bond < 100')
print(classed.bond_amount.loc[classed.bond_amount < 10].count(), 'out of', len(classed), 'have bond < 10')
print(classed.bond_amount.median(), 'median bond')

7000000 max bond
1 min bond
93 out of 28808 have bond < 100
16 out of 28808 have bond < 10
25400.0 median bond


In [11]:
# Save the cleaned dataframes to csv for unit 7, since it's a new notebook.
classed.to_csv('detainees_classed_offenses.csv', index=False)
not_classed.to_csv('detainees_unclassed_offenses.csv', index=False)

In [12]:
# Just print statements to again see some basic information.
print(not_classed.offense.nunique(), 'offenses without classifications')
#print(not_classed.offense.unique().tolist())
print(not_classed['offense'].loc[not_classed['offense'] == 'VIOLATION OF PROBATION OR COND DISCHG'].count(), 'probation violations')

88 offenses without classifications
6757 probation violations


In [13]:
# Some basic math calculations to give an idea of scope.
print(len(classed)/classed.identifier.nunique(), 'stays per detainee with classified offense')
print(classed.identifier.nunique(), 'unique detainees classified')
#print(classed.latest_admission_date.loc[classed.latest_admission_date < '2014-01-01'])

print(grouped_df.identifier.nunique(), 'unique detainees in whole')
print(not_classed.identifier.nunique(), 'unique detainees not classified')

1.2711468031593347 stays per detainee with classified offense
22663 unique detainees classified
29531 unique detainees in whole
9808 unique detainees not classified


In [14]:
print(df.download_date.nunique())
classed[classed['latest_admission_date'] < '2015-01-01'].nunique()

798


download_date            369
identifier               803
latest_admission_date    644
race                       5
gender                     2
age                       49
bond_amount              156
offense                  100
facility                  18
detainer                   6
offense_class             10
days                     738
counts                   315
dtype: int64

In [15]:
df.loc[df.identifier == 'ZZEBBBJW']

,download_date,identifier,latest_admission_date,race,gender,age,bond_amount,offense,facility,detainer,offense_class,days,counts
163282,2016-09-07,ZZEBBBJW,2016-08-24,WHITE,M,43,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,14 days,26
166656,2016-09-08,ZZEBBBJW,2016-08-24,WHITE,M,43,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,15 days,26
170001,2016-09-09,ZZEBBBJW,2016-08-24,WHITE,M,43,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,16 days,26
173349,2016-09-12,ZZEBBBJW,2016-08-24,WHITE,M,43,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,19 days,26
176698,2016-09-13,ZZEBBBJW,2016-08-24,WHITE,M,43,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,20 days,26
180274,2016-09-14,ZZEBBBJW,2016-08-24,WHITE,M,43,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,21 days,26
184072,2016-09-15,ZZEBBBJW,2016-08-24,WHITE,M,43,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,22 days,26
188593,2016-09-16,ZZEBBBJW,2016-08-24,WHITE,M,43,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,23 days,26
191120,2016-09-19,ZZEBBBJW,2016-08-24,WHITE,M,44,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,26 days,26
196126,2016-10-04,ZZEBBBJW,2016-08-24,WHITE,M,44,15000,INJURY OR RISK OF INJURY TO MINOR F,NEW HAVEN CC,NONE,F,41 days,26
